Combining Tree

In [15]:
%%writefile CombiningTree.cs
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;
using System.Collections;
using System.Threading;

namespace CombiningTree
{
    class CombiningTree
    {
        Node[] leaf;
        Node[] nodes;

        public CombiningTree(int width)
        {
            nodes = new Node[2 * width - 1];
            nodes[0] = new Node();
            for (int i = 1; i < nodes.Length; i++)
            {
                nodes[i] = new Node(nodes[(i - 1) / 2]);
            }
            leaf = new Node[width];
            for (int i = 0; i < leaf.Length; i++)
            {
                leaf[i] = nodes[nodes.Length - i - 1];
            }
        }

        static long Id()
        {
            return Thread.CurrentThread.ManagedThreadId;
        }

        public int getAndIncrement(int id)
        {
            Stack<Node> stack = new Stack<Node>();
            Node myLeaf = leaf[id % leaf.Length];
            Node node = myLeaf;
            // precombining phase
            try
            {
                while (node.precombine())
                {
                    node = node.parent;
                }
            }
            catch (Exception e) { Console.WriteLine(e); Console.ReadKey(); }
            Node stop = node;
            // combining phase
            int combined = 1;
            for (node = myLeaf; node != stop; node = node.parent)
            {
                try { combined = node.combine(combined); } catch (Exception) { }
                stack.Push(node);
            }


            // operation phase
            int prior = stop.op(combined);
            // distribution phase
            while (stack.Count > 0)
            {
                node = stack.Pop();
                node.distribute(prior);
            }
            return prior;


        }

        public int get()
        {
            return nodes[0].result;
        }
    }
}


Writing CombiningTree.cs


In [16]:
%%writefile Node.cs
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;
using System.Diagnostics;
using System.Threading;

namespace CombiningTree
{
	class Node
	{
		enum CStatus { IDLE, FIRST, SECOND, RESULT, ROOT };
		bool locked;
		CStatus cStatus;
		int firstValue, secondValue;
		public int result;
		public Node parent;

		public Node()
		{
			cStatus = CStatus.ROOT;
			locked = false;
		}
		public Node(Node myParent)
		{
			parent = myParent;
			cStatus = CStatus.IDLE;
			locked = false;
		}

		public bool precombine()
		{
			lock (this)
			{
				while (locked) Monitor.Wait(this);

				switch (cStatus)
				{
					case CStatus.IDLE:
						cStatus = CStatus.FIRST;
						return true;
					case CStatus.FIRST:
						locked = true;
						cStatus = CStatus.SECOND;
						return false;
					case CStatus.ROOT:
						return false;
					default:
						throw new Exception("unexpected Node state" + cStatus);
				}
			}
		}

		public int combine(int combined)
		{
			lock (this)
			{
				while (locked) Monitor.Wait(this);

				locked = true;
				firstValue = combined;
				switch (cStatus)
				{
					case CStatus.FIRST:
						return firstValue;
					case CStatus.SECOND:
						return firstValue + secondValue;
					default:
						throw new Exception("unexpected Node state " + cStatus);
				}
			}
		}

		public int op(int combined)
		{
			lock (this)
			{
				switch (cStatus)
				{
					case CStatus.ROOT:
						int prior = result;
						result += combined;
						return prior;
					case CStatus.SECOND:
						secondValue = combined;
						locked = false;

						Monitor.PulseAll(this); // wake up waiting threads

						while (cStatus != CStatus.RESULT) Monitor.Wait(this);
						locked = false;
						Monitor.PulseAll(this);

						cStatus = CStatus.IDLE;
						return result;
					default:
						throw new Exception("unexpected Node state");
				}
			}

		}

		public void distribute(int prior)
		{
			lock (this)
			{
				switch (cStatus)
				{
					case CStatus.FIRST:
						cStatus = CStatus.IDLE;
						locked = false;
						break;
					case CStatus.SECOND:
						result = prior + firstValue;
						cStatus = CStatus.RESULT;
						break;
					default:
						throw new Exception("unexpected Node state");
				}
				Monitor.PulseAll(this);
			}


		}
	}
}


Writing Node.cs


In [17]:
%%writefile Test.cs
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;
using System.Threading;
using System.Diagnostics;

namespace CombiningTree
{
    class Test
    {
        static CombiningTree tree;
        static int TH = 12, NUM = 100;
        static int width;
        // tree: combining tree where threads do increment ops
        // TH: number of threads
        // NUM: number of increment ops each thread performs

        static void Main(String[] args)
        {
            width = (int) Math.Ceiling((decimal)TH / 2);
            tree = new CombiningTree(width);
            var stopWatch = new Stopwatch();


            Log(width + "-leaves Combining tree.");
            Log("Starting " + TH + " threads doing increments ...");

            stopWatch.Start();

            Parallel.For(0, TH, (index) =>
            {   
                DateTime start = DateTime.Now;
                for (int i = 0; i < NUM; i++)
                   tree.getAndIncrement(index);
                DateTime stop = DateTime.Now;
                Log(index + ": done in " + (stop - start).TotalSeconds + "s");
            });

            stopWatch.Stop();
            var ts = stopWatch.Elapsed;
            Log("Total: " + tree.get());
            Log("Total time: " + String.Format("{0:00}.{1:00}", ts.Seconds, ts.Milliseconds / 10)+'s');

            //Console.ReadKey();
        }

        static void Log(String x)
        {
            Console.WriteLine(x);
        }

}
}



Overwriting Test.cs


In [18]:
!csc *.cs

Microsoft (R) Visual C# Compiler version 3.9.0-6.21160.10 (59eedc33)
Copyright (C) Microsoft Corporation. All rights reserved.



In [19]:
!Test.exe

6-leaves Combining tree.
Starting 12 threads doing increments ...
5: done in 0.0140028s
11: done in 0.0140028s
10: done in 0.0140028s
4: done in 0.0149982s
0: done in 0.016001s
2: done in 0.0149982s
8: done in 0.0149982s
6: done in 0.0149982s
7: done in 0.016s
3: done in 0.016s
1: done in 0.0170028s
9: done in 0.016s
Total: 1200
Total time: 00.02s


Counting Network

In [1]:
%%writefile Balancer.cs
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;
using System.Threading;

namespace BitonicCountingNetwork
{
    public class Balancer
    {
        private int _toggle = 1;

        public int Traverse()
        {
            while (true)
            {
                if (1 == Interlocked.Exchange(ref _toggle, 0))
                {
                    return 0;
                }
                if (0 == Interlocked.Exchange(ref _toggle, 1))
                {
                    return 1;
                }
            }
        }
    }
}


Writing Balancer.cs


In [2]:
%%writefile Bitonic.cs
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;

namespace BitonicCountingNetwork
{
    public class Bitonic
    {
        private Bitonic[] _half;

        private Merger _merger;

        private readonly int _width;

        public Bitonic(int width)
        {
            _width = width;

            _merger = new Merger(_width);

            if (_width > 2)
            {
                _half = new Bitonic[]
                {
                    new Bitonic(_width / 2),
                    new Bitonic(_width / 2)
                };
            }
        }

        public int Traverse(int input)
        {
            int output = 0;

            if (_width > 2)
            {
                output = _half[input / (_width / 2)].Traverse(input / 2);
            }
            return _merger.Traverse((input >= (_width / 2) ? (_width / 2) : 0) + output);
        }
    }
}


Writing Bitonic.cs


In [3]:
%%writefile Merger.cs
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;

namespace BitonicCountingNetwork
{
    class Merger
    {
        private Merger[] _half;

        private Balancer[] _layer;

        private readonly int _width;

        public Merger(int width)
        {
            _width = width;

            _layer = new Balancer[width / 2];
            for (var i = 0; i < width / 2; ++i)
            {
                _layer[i] = new Balancer();
            }

            if (_width > 2)
            {
                _half = new Merger[]
                {
                    new Merger(width / 2),
                    new Merger(width / 2)
                };
            }
        }

        public int Traverse(int input)
        {
            var output = 0;

            if (_width <= 2) return _layer[0].Traverse();

            if (input < _width / 2)
            {
                output = _half[input % 2].Traverse(input / 2);
            }
            else
            {
                output = _half[1 - (input % 2)].Traverse(input / 2);
            }

            return (2 * output) + _layer[output].Traverse();
        }
    }
}


Writing Merger.cs


In [11]:
%%writefile Test.cs
using System;
using System.Threading.Tasks;
using System.Diagnostics;
using System.Threading;

namespace BitonicCountingNetwork
{
    class Test
    {
        static void Main(string[] args)
        {
            const int width = 4;
            var bitonic = new Bitonic(width);
            var counters = new int[width];

            const int tokenCount = 1000;
            var tokens = new int[tokenCount];

            var rand = new Random();
            var randLock = new object();
            var stopWatch = new Stopwatch();

            Parallel.For(0, tokenCount, (i) =>
            {
                int next;
                lock (randLock)
                {
                    next = rand.Next(width);
                }
                tokens[i] = next;
            });

            stopWatch.Start();
            Parallel.For(0, tokenCount, (i) =>
            {
                Interlocked.Increment(ref counters[bitonic.Traverse(tokens[i])]);
            });
            stopWatch.Stop();

            var traversing = stopWatch.Elapsed.TotalSeconds;

            for (var i = 0; i < width; ++i)
            {
                Console.WriteLine($"Output: {i} Count: {counters[i]}");
            }

            Console.WriteLine($"Time to traverse the network: {traversing.ToString()} s");

            //Console.ReadKey();
        }
    }
}


Overwriting Test.cs


In [12]:
!csc *.cs

Microsoft (R) Visual C# Compiler version 3.9.0-6.21160.10 (59eedc33)
Copyright (C) Microsoft Corporation. All rights reserved.



In [13]:
!Test.exe

Output: 0 Count: 250
Output: 1 Count: 250
Output: 2 Count: 250
Output: 3 Count: 250
Time to traverse the network: 0.0006264 s
